In [1]:
from data_utils import SeparatedMelHarmTextDataset, MelHarmTextCollatorForSeq2Seq
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
import torch
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn as nn
from transformers import RobertaModel, RobertaTokenizer

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = '/media/maindisk/maximos/data/hooktheory_train'

In [3]:
chordSymbolTokenizer = ChordSymbolTokenizer.from_pretrained('saved_tokenizers/ChordSymbolTokenizer')
rootTypeTokenizer = RootTypeTokenizer.from_pretrained('saved_tokenizers/RootTypeTokenizer')
pitchClassTokenizer = PitchClassTokenizer.from_pretrained('saved_tokenizers/PitchClassTokenizer')
rootPCTokenizer = RootPCTokenizer.from_pretrained('saved_tokenizers/RootPCTokenizer')
melodyPitchTokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')

In [4]:
m_chordSymbolTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, chordSymbolTokenizer)
m_rootTypeTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootTypeTokenizer)
m_pitchClassTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, pitchClassTokenizer)
m_rootPCTokenizer = MergedMelHarmTokenizer(melodyPitchTokenizer, rootPCTokenizer)

In [5]:
tokenizer = m_chordSymbolTokenizer
tokenizer_name = 'ChordSymbolTokenizer'

description_modes = [
    'specific_chord',
    'chord_root',
    'pitch_class'
]

# roberta_model = "roberta-base"
# text_tokenizer = RobertaTokenizer.from_pretrained(roberta_model)

dataset = SeparatedMelHarmTextDataset(root_dir, tokenizer, max_length=512, num_bars=64, \
    description_mode=description_modes[2])
# Data collator for BART
def create_data_collator(tokenizer, model):
    return MelHarmTextCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [7]:
print(dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'harmony_input_ids', 'labels', 'txt'])


/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

bart = BartForConditionalGeneration(bart_config)

bart_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'
if device == 'cpu':
    checkpoint = torch.load(bart_path, map_location="cpu", weights_only=True)
else:
    checkpoint = torch.load(bart_path, weights_only=True)
bart.load_state_dict(checkpoint)

bart.to(device)
bart.eval()

bart_encoder, bart_decoder = bart.get_encoder(), bart.get_decoder()
bart_encoder.to(device)
bart_decoder.to(device)

# Freeze BART parameters
for param in bart_encoder.parameters():
    param.requires_grad = False
for param in bart_encoder.parameters():
    param.requires_grad = False

/home/maximos/.local/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
collator = create_data_collator(tokenizer, model=bart)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collator)

In [10]:
b = next(iter(dataloader))

/home/maximos/.local/lib/python3.11/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [11]:
print(b['txt'])

['Bar number 1 begins with a chord with a F pitch class.', 'Bar number 6 begins with a chord with a C pitch class.', 'Bar number 14 begins with a chord with a D pitch class.', 'Bar number 4 begins with a chord with a E pitch class.', 'Bar number 1 begins with a chord with a D pitch class.', 'Bar number 0 begins with a chord with a A pitch class.', 'Bar number 5 begins with a chord with a G pitch class.', 'Bar number 12 begins with a chord with a C pitch class.', 'Bar number 5 begins with a chord with a E pitch class.', 'Bar number 5 begins with a chord with a G pitch class.', 'Bar number 1 begins with a chord with a G pitch class.', 'Bar number 5 begins with a chord with a G pitch class.', 'Bar number 1 begins with a chord with a A pitch class.', 'Bar number 1 begins with a chord with a E pitch class.', 'Bar number 3 begins with a chord with a F pitch class.', 'Bar number 7 begins with a chord with a E pitch class.', 'Bar number 7 begins with a chord with a F# pitch class.', 'Bar numbe

In [12]:
from models import TransTextVAE

In [13]:
model = TransTextVAE(bart, tokenizer=tokenizer, device=device)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/maximos/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


TransTextVAE(
  (transformer): BartForConditionalGeneration(
    (model): BartModel(
      (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(514, 512)
        (layers): ModuleList(
          (0-7): 8 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=512, out_features=512, bias=True)
            (fc2): Linear(in_featu

In [14]:
y = model(b['input_ids'].to(device), b['txt'])

In [15]:
print(y.keys())

dict_keys(['loss', 'recon_loss', 'kl_loss', 'x', 'recon_x', 'generated_ids', 'generated_recon_ids', 'generated_markov', 'recon_markov'])
